In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

import pandas as pd
import numpy as np

In [86]:
# load the csv data file
df = pd.read_csv("./data/nci60.csv")
# X = np.array(df.iloc[:,2:])
# y = np.array(df.iloc[:,1])
X = df.iloc[:,2:]
y = df.iloc[:,1]
print(len(y))

60


In [26]:
def rfe_rf_tim(X, y, num_feature_to_selected):
    from sklearn.ensemble import RandomForestRegressor
    import time
    rf = RandomForestRegressor()
    X_new = X
    record = {}
    keep_ = X.shape[1]-num_feature_to_selected
    start_time = time.time()
    for i in range(keep_):
        # get the column name
        names = list(X_new)
        # fit the random forest and get the weights of features
        fit = rf.fit(X_new, y)
        # remove the feature with the smallest weight
        X_new = X_new.drop(X_new.columns[[np.argmin(fit.feature_importances_)]], 1)
        # record the index and value of the deleted feature
        record.update({str(names[np.argmin(fit.feature_importances_)]):fit.feature_importances_[np.argmin(fit.feature_importances_)]})
    elapsed_time = time.time() - start_time
    return X_new, record, elapsed_time

In [30]:
# Mean decrease impurity
rf = RandomForestRegressor()
rf.fit(X.iloc[:,2:100], y)
names = list(X.iloc[:,2:100])
print("Features sorted by their score:")
print(sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), names), 
             reverse=True))

Features sorted by their score:
[(0.1933, 'ABHD5'), (0.1328, 'ABHD14A'), (0.075999999999999998, 'ABCA4'), (0.071199999999999999, 'ABL1'), (0.039399999999999998, 'ABCA10'), (0.0385, 'ABCC10'), (0.032199999999999999, 'ABRACL'), (0.023800000000000002, 'ACACB'), (0.021700000000000001, 'ABCB1'), (0.020899999999999998, 'ABHD12'), (0.018100000000000002, 'A2ML1'), (0.017399999999999999, 'ACAD10'), (0.016899999999999998, 'ABCA12'), (0.015299999999999999, 'ABCA8'), (0.0147, 'AASS'), (0.0144, 'ABCC1'), (0.0138, 'ACADL'), (0.0137, 'ABCG8'), (0.0126, 'ACADSB'), (0.010200000000000001, 'ABCA9'), (0.01, 'ABRA'), (0.0099000000000000008, 'ABCC11'), (0.0099000000000000008, 'ABCA7'), (0.0094999999999999998, 'ABCG5'), (0.0094999999999999998, 'ABCC12'), (0.0094999999999999998, 'AASDH'), (0.0091999999999999998, 'AADACL3'), (0.0085000000000000006, 'ABCC4'), (0.0073000000000000001, 'ABCC2'), (0.0067999999999999996, 'ABCA1'), (0.0066, 'ABCC9'), (0.0064000000000000003, 'ABCC6'), (0.0055999999999999999, 'ABP1'), 

In [90]:
# Mean decrease accuracy
from sklearn.cross_validation import ShuffleSplit
from sklearn.metrics import r2_score
from collections import defaultdict

rf = RandomForestRegressor()
scores = defaultdict(list)
X = X.iloc[:,2:500]

#crossvalidate the scores on a number of different random splits of the data
for train_idx, test_idx in ShuffleSplit(len(X), 5, .3):
    #train_idx = train_idx.tolist()
    #test_idx = test_idx.tolist()
    X_train = X.iloc[train_idx,:]
    X_test = X.iloc[test_idx,:]
    y_train = y[train_idx]
    y_test = y[test_idx]
    r = rf.fit(X_train, y_train)
    acc = r2_score(y_test, rf.predict(X_test))
    for i in range(X.shape[1]):
        X_t = X_test
        # np.random.shuffle(X_t.iloc[:,i])
        shuff_acc = r2_score(y_test, rf.predict(X_t))
        scores[names[i]].append((acc-shuff_acc)/acc)
print("Features sorted by their score:")
print(sorted([(round(np.mean(score), 4), feat) for
              feat, score in scores.items()], reverse=True))


Features sorted by their score:
[(0.0, 'ACAP1'), (0.0, 'ACADVL'), (0.0, 'ACADSB'), (0.0, 'ACADS'), (0.0, 'ACADM'), (0.0, 'ACADL'), (0.0, 'ACAD8'), (0.0, 'ACAD11'), (0.0, 'ACAD10'), (0.0, 'ACACB'), (0.0, 'ACACA'), (0.0, 'ACAA2'), (0.0, 'ACAA1'), (0.0, 'ABTB2'), (0.0, 'ABTB1'), (0.0, 'ABRACL'), (0.0, 'ABRA'), (0.0, 'ABP1'), (0.0, 'ABLIM3'), (0.0, 'ABLIM1'), (0.0, 'ABL2'), (0.0, 'ABL1'), (0.0, 'ABI3'), (0.0, 'ABI2'), (0.0, 'ABI1'), (0.0, 'ABHD8'), (0.0, 'ABHD6'), (0.0, 'ABHD5'), (0.0, 'ABHD3'), (0.0, 'ABHD2'), (0.0, 'ABHD16B'), (0.0, 'ABHD15'), (0.0, 'ABHD14A'), (0.0, 'ABHD12B'), (0.0, 'ABHD12'), (0.0, 'ABHD1'), (0.0, 'ABCG8'), (0.0, 'ABCG5'), (0.0, 'ABCG4'), (0.0, 'ABCG2'), (0.0, 'ABCG1'), (0.0, 'ABCF2'), (0.0, 'ABCD4'), (0.0, 'ABCD3'), (0.0, 'ABCD2'), (0.0, 'ABCD1'), (0.0, 'ABCC9'), (0.0, 'ABCC8'), (0.0, 'ABCC6'), (0.0, 'ABCC4'), (0.0, 'ABCC3'), (0.0, 'ABCC2'), (0.0, 'ABCC12'), (0.0, 'ABCC11'), (0.0, 'ABCC10'), (0.0, 'ABCC1'), (0.0, 'ABCB9'), (0.0, 'ABCB6'), (0.0, 'ABCB5'), (0.0, 'ABCB4

In [72]:
a = [14,55, 28, 44, 22]
type(a)
# X.iloc[a,:]
for train_idx, test_idx in ShuffleSplit(len(X), 2, .3):
    train_idx = train_idx.tolist()
    test_idx = test_idx.tolist()
    X_train = X.iloc[train_idx,:]
    X_test = X.iloc[test_idx,:]
    y_train = y[train_idx]
    y_test = y[test_idx]
    print(y_train)

52    6.083240
8     4.101435
25    5.886417
24    6.684804
17    5.813246
46    6.168251
56    6.092411
51    6.064225
2     5.214774
49    6.163438
28    5.035801
11    5.757420
34    5.407344
53    5.174760
57    5.869474
33    5.925060
9     5.839177
48    6.228172
55    6.205731
18    6.825297
29    5.441209
35    5.668670
39    5.952971
58    5.969394
13    6.254466
42    6.066126
43    5.242544
31    6.240240
44    6.103252
47    6.141367
3     5.534106
30    5.675730
10    5.310726
27    5.999890
4     5.733321
36    6.109751
12    6.026118
41    6.180748
32    6.018799
26    6.225627
19    5.986424
20    6.435436
Name: IC50, dtype: float64
38    5.370651
28    5.035801
15    5.919628
29    5.441209
39    5.952971
34    5.407344
51    6.064225
49    6.163438
5     5.931407
44    6.103252
4     5.733321
11    5.757420
19    5.986424
3     5.534106
25    5.886417
17    5.813246
14    5.958878
41    6.180748
8     4.101435
10    5.310726
9     5.839177
47    6.141367
33    5.92506

In [27]:
X_new, record, elapsed_time = rfe_rf_tim(X.iloc[:,2:50], y, 25)
for key,value in record.items(): 
    print(key,':',value)

X_new
#elapsed_time

AASDHPPT : 0.0
ABCD1 : 0.0
AARSD1 : 0.0
AAGAB : 0.0
AAMP : 0.00263407546055
AATF : 0.0
ABCB6 : 0.00489942258756
ABCB4 : 0.0
ABCD2 : 0.000232389392413
ABCB10 : 0.000978858006532
A4GALT : 0.0
AAED1 : 0.00617003249863
ABCA1 : 0.00115011197232
AACS : 0.00144961772637
ABCA5 : 0.000119852606899
AARS2 : 0.00401676518806
A4GNT : 0.0
AADAT : 0.0
ABCA6 : 0.0
ABCA2 : 0.00153036762688
AADACL2 : 0.0
AASDH : 1.29166335244e-05
ABCC8 : 0.00253851201986


,A2ML1,AAAS,AADACL3,AADACL4,AARS,AASS,ABCA10,ABCA12,ABCA3,ABCA4,...,ABCB9,ABCC1,ABCC10,ABCC11,ABCC12,ABCC2,ABCC3,ABCC4,ABCC6,ABCC9
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
7,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
8,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
from sklearn.model_selection import KFold, cross_val_score
k_fold = KFold(n_splits=5)
rf = RandomForestRegressor()
# model = clf.set_params(kernel='linear')
score = cross_val_score(rf, X_new, y, cv=k_fold, n_jobs=-1)
np.mean(score)

-0.82997179637651686

In [11]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.svm import SVR
k_fold = KFold(n_splits=5)
clf = SVR()
# model = clf.set_params(kernel='linear')
score = cross_val_score(clf, X_new, y, cv=k_fold, n_jobs=-1)
np.mean(score)

-0.39388203872926386